In [0]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import torch
from torch import nn
from torch import optim
import torchvision
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import time as time
from PIL import ImageFile
import numpy as np
import pandas as pd
ImageFile.LOAD_TRUNCATED_IMAGES = True

use_cuda = torch.cuda.is_available()

In [3]:
use_cuda

True

this project is done in colab <br>
Datset is preprocessed according to the need and upload to the drive <br>
intial structure of dataset is <br>
Breakhis/ ,<br>
&emsp; benign/ <br>
&emsp; &emsp;class0/images.. <br>
&emsp; &emsp;class1/images.. <br>
&emsp; &emsp;class2/images.. <br>
&emsp; &emsp;class3/images.. <br>
&emsp; malignant/ <br>
&emsp; &emsp;class4/images.. <br>
&emsp; &emsp;class5/images.. <br>
&emsp; &emsp;class6/images.. <br>
&emsp; &emsp;class7/images.. <br>


After running the pred_data.py , dataset structure is <br>
Breakhis/ ,<br>
&emsp; train/ <br>
&emsp; &emsp;class0/images.. <br>
&emsp; &emsp;class1/images.. <br>
&emsp; &emsp;class2/images.. <br>
&emsp; &emsp;class3/images.. <br>
&emsp; &emsp;class4/images.. <br>
&emsp; &emsp;class5/images.. <br>
&emsp; &emsp;class6/images.. <br>
&emsp; &emsp;class7/images.. <br>
&emsp; test/ <br>
&emsp; &emsp;class0/images.. <br>
&emsp; &emsp;class1/images.. <br>
&emsp; &emsp;class2/images.. <br>
&emsp; &emsp;class3/images.. <br>
&emsp; &emsp;class4/images.. <br>
&emsp; &emsp;class5/images.. <br>
&emsp; &emsp;class6/images.. <br>
&emsp; &emsp;class7/images.. <br>
&emsp; valid/ <br>
&emsp; &emsp;class0/images.. <br>
&emsp; &emsp;class1/images.. <br>
&emsp; &emsp;class2/images.. <br>
&emsp; &emsp;class3/images.. <br>
&emsp; &emsp;class4/images.. <br>
&emsp; &emsp;class5/images.. <br>
&emsp; &emsp;class6/images.. <br>
&emsp; &emsp;class7/images.. <br>

In [2]:
#mount drive in colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!unzip /content/drive/'My Drive'/prep_data1.zip

Archive:  /content/drive/My Drive/prep_data1.zip
replace prep_data1/test/A/SOB_B_A-14-22549AB-100-001.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace prep_data1/test/A/SOB_B_A-14-22549AB-100-009.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
#path
data_dir = '/content/prep_data1/'

# Define transforms for the training data and testing data
train_transforms=transforms.Compose([transforms.Resize(256),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomVerticalFlip(),
                                    transforms.RandomRotation(15),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.5, 0.5, 0.5],
                                                        [0.5, 0.5, 0.5])
                                   ])
#transformations for validation and test  datasets
valid_transforms=transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.5, 0.5, 0.5],
                                                        [0.5, 0.5, 0.5])
                                    ])


train_data=datasets.ImageFolder(os.path.join(data_dir,'train'),transform=train_transforms)
valid_data=datasets.ImageFolder(os.path.join(data_dir,'valid'),transform=valid_transforms)
test_data=datasets.ImageFolder(os.path.join(data_dir,'test'),transform=valid_transforms)
classes=train_data.classes
print(classes)

['A', 'DC', 'F', 'LC', 'MC', 'PC', 'PT', 'TA']


Number of samples in each class [887, 6130, 480, 317, 972, 101, 128] <br>
dataset is Imbalanced <br>
in order to deal with imbalance data set i am using WeightedRandomSampler() from pytorch

In [0]:
#count the number of samples in each image
sample_counts=[] 
for i in [train_data.imgs,test_data.imgs,valid_data.imgs]:
    for item in i:
        sample_counts.append(item[1])
class_counts=dict(pd.Series(sample_counts).value_counts())

In [0]:
#Get weights to each and images
class_sample_count=[class_counts[i] for i in sorted(class_counts)]
weights = 1 / torch.Tensor(class_sample_count)
weights = weights.double()
train_targets = [sample[1] for sample in train_data.imgs]
train_samples_weight = [weights[class_id] for class_id in train_targets]

In [0]:
#from torchsampler import ImbalancedDatasetSampler
from torch.utils.data.sampler import WeightedRandomSampler
batch_size=64

#loding the data into loaders
train_Loader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,sampler=WeightedRandomSampler(train_samples_weight, len(train_data)),shuffle=False,num_workers=4)
valid_Loader=torch.utils.data.DataLoader(valid_data,batch_size=batch_size,shuffle=False,num_workers=4)
test_Loader=torch.utils.data.DataLoader(test_data,batch_size=batch_size,shuffle=False,num_workers=4)

# small dataset , different data -transfer learning
In our case dataset is small, so todeal with small datasets is a good to remove the high level feature layers from pretrained model. <br>

In this project i have used vgg16(),<br>
## fine-tune
removed the last convolutional block of vgg16 i.e last 3convolutional layers followed by action and maxpooling
and add classifier 

In [8]:
class vgg_net(nn.Module):
    def __init__(self):
        super().__init__()
        vgg_model=models.vgg16(pretrained=True)
        self.features=nn.Sequential(*list(vgg_model.features)[:-7])
        self.classifier=nn.Sequential(nn.Linear(100352,4096),*list(vgg_model.classifier[1:-1]),nn.Linear(4096,8))
        
    def forward(self,x):
        x=self.features(x)
        x = x.view(-1, 512 * 14 * 14)
        x=self.classifier(x)
        
        return x
model=vgg_net()
model        

vgg_net(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilati

In [0]:
for param in model.features.parameters():
    param.requires_grad = False

#model.classifier[0].in_features=100352
if use_cuda:
    model = model.cuda()

# mention the criterion and optimizers and their parameters

In [0]:
#mention the criterion and optimizers and their parameters
from torch.optim.lr_scheduler import StepLR,ReduceLROnPlateau
criterion= nn.CrossEntropyLoss()
optimizer= optim.SGD(model.parameters(),lr=0.007,momentum=0.9,weight_decay=0.1, nesterov=True)
#scheduler = StepLR(optimizer_transfer, step_size=2, gamma=0.2)
scheduler=ReduceLROnPlateau(optimizer, mode='max', factor=0.7, patience=1, verbose=True)

In [0]:
torch.cuda.empty_cache()

# train and save the model

In [12]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda,save_path):
    valid_loss_min = np.Inf
    for epoch in range(1, n_epochs+1):

        # keep track of training and validation loss
        train_loss = 0.0
        valid_loss=0.0
        correct=0.0
        # model by default is set to train
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move tensors to GPU if CUDA is available
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss 
            train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            _,pred=torch.max(output,1)
            correct+=torch.sum(pred.eq(target.data.view_as(pred)))
            
            
        # validate the model 
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            with torch.no_grad():
                output=model(data)
            loss=criterion(output.data,target)
            #v_total+=data.size(0)
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
        
            
        # print training/validation statistics 
        print('Epoch: {}  Training Loss: {:.6f}  Accuracy {:.6f}  Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            correct/len(train_data),
            valid_loss,
            ))
        scheduler.step(valid_loss)
        #save model
        if valid_loss < valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    
    return model

loaders={'train':train_Loader,
         'valid':valid_Loader,
         'test':test_Loader
        }
model = train(50, loaders, model, optimizer, criterion, use_cuda, 'vgg_model_fine_tune.pt')

# load the model that got the best validation accuracy
model.load_state_dict(torch.load('vgg_model_fine_tune.pt'))

Epoch: 1  Training Loss: 1.892627  Accuracy 0.241089  Validation Loss: 1.730102
Validation loss decreased (inf --> 1.730102). Saving model...
Epoch: 2  Training Loss: 1.318676  Accuracy 0.498156  Validation Loss: 1.279369
Validation loss decreased (1.730102 --> 1.279369). Saving model...
Epoch: 3  Training Loss: 1.079068  Accuracy 0.604741  Validation Loss: 1.157873
Epoch     3: reducing learning rate of group 0 to 4.9000e-03.
Validation loss decreased (1.279369 --> 1.157873). Saving model...
Epoch: 4  Training Loss: 0.908948  Accuracy 0.668306  Validation Loss: 1.190614
Epoch: 5  Training Loss: 0.870743  Accuracy 0.687270  Validation Loss: 1.298284
Epoch     5: reducing learning rate of group 0 to 3.4300e-03.
Epoch: 6  Training Loss: 0.758732  Accuracy 0.733802  Validation Loss: 1.059741
Validation loss decreased (1.157873 --> 1.059741). Saving model...
Epoch: 7  Training Loss: 0.703472  Accuracy 0.758736  Validation Loss: 0.964628
Epoch     7: reducing learning rate of group 0 to 2.4

<All keys matched successfully>

# test the model 

In [13]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    confusion_matrix = torch.zeros(8, 8)

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
        for t, p in zip(target.view(-1), pred.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
            
    print('Test Loss: {:.6f} \n confusion matrix: \n {}'.format(test_loss,confusion_matrix))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
    return confusion_matrix

# call test function    
confusion_matrix=test(loaders, model, criterion, use_cuda)

Test Loss: 0.780660 
 confusion matrix: 
 tensor([[ 82.,   0.,   4.,   1.,   3.,   1.,   1.,   0.],
        [  7., 483.,  10., 131.,  33.,  39.,   2.,   7.],
        [  9.,   9., 119.,   8.,   2.,   1.,  20.,  18.],
        [  3.,  19.,   0.,  83.,   4.,   2.,   0.,   0.],
        [  5.,  28.,   3.,  12., 111.,   4.,   0.,   9.],
        [  0.,  15.,   7.,   1.,   1.,  78.,   1.,   5.],
        [  6.,   1.,  11.,   0.,   1.,   4.,  64.,   6.],
        [  4.,   3.,   6.,   0.,   1.,   0.,   1.,  93.]])

Test Accuracy: 70% (1113/1582)


# visualising model using tensorboard

In [0]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/breast_cancer')

In [0]:
class_probs = []
class_preds = []
model.eval()
with torch.no_grad():
    for data in loaders['test']:
        images, labels = data
        if use_cuda:
          images=images.cuda()
        output = model(images)
        class_probs_batch = [F.softmax(el, dim=0) for el in output]
        _, class_preds_batch = torch.max(output, 1)

        class_probs.append(class_probs_batch)
        class_preds.append(class_preds_batch)

test_probs = torch.cat([torch.stack(batch) for batch in class_probs])
test_preds = torch.cat(class_preds)

# helper function
def add_pr_curve_tensorboard(class_index, test_probs, test_preds, global_step=0):
    '''
    Takes in a "class_index" from 0 to 9 and plots the corresponding
    precision-recall curve
    '''
    tensorboard_preds = test_preds == class_index
    tensorboard_probs = test_probs[:, class_index]

    writer.add_pr_curve(classes[class_index],
                        tensorboard_preds,
                        tensorboard_probs,
                        global_step=global_step)
    writer.close()

# plot all the pr curves
for i in range(len(classes)):
    add_pr_curve_tensorboard(i, test_probs, test_preds)

In [ ]:
!tensorboard --logdir=runs --host localhost

# evaluation metrics

sensitivity: TP/(TP+FN) <br>
specifity :(TN / (TN+FP)

In [35]:
nb_classes=8
TP = confusion_matrix.diag()
for c in range(nb_classes):
    idx = torch.ones(nb_classes).byte()
    idx[c] = 0
    # all non-class samples classified as non-class
    TN = confusion_matrix[idx.nonzero()[:, None], idx.nonzero()].sum() #conf_matrix[idx[:, None], idx].sum() - conf_matrix[idx, c].sum()
    # all non-class samples classified as class
    FP = confusion_matrix[idx, c].sum()
    # all class samples not classified as class
    FN = confusion_matrix[c, idx].sum()
    sensitivity = (TP[c] / (TP[c]+FN))
    specificity = (TN / (TN+FP))

    print('Class {}\nTP {}, TN {}, FP {}, FN {}'.format(
            c, TP[c], TN, FP, FN))
    print('Sensitivity = {}'.format(sensitivity))
    print('Specificity = {}'.format(specificity))

Class 0
TP 82.0, TN 1456.0, FP 34.0, FN 10.0
Sensitivity = 0.8913043737411499
Specificity = 0.9771811962127686
Class 1
TP 483.0, TN 795.0, FP 75.0, FN 229.0
Sensitivity = 0.6783707737922668
Specificity = 0.9137930870056152
Class 2
TP 119.0, TN 1355.0, FP 41.0, FN 67.0
Sensitivity = 0.6397849321365356
Specificity = 0.9706303477287292
Class 3
TP 83.0, TN 1318.0, FP 153.0, FN 28.0
Sensitivity = 0.7477477192878723
Specificity = 0.8959891200065613
Class 4
TP 111.0, TN 1365.0, FP 45.0, FN 61.0
Sensitivity = 0.645348846912384
Specificity = 0.9680851101875305
Class 5
TP 78.0, TN 1423.0, FP 51.0, FN 30.0
Sensitivity = 0.7222222089767456
Specificity = 0.9654002785682678
Class 6
TP 64.0, TN 1464.0, FP 25.0, FN 29.0
Sensitivity = 0.6881720423698425
Specificity = 0.9832102060317993
Class 7
TP 93.0, TN 1429.0, FP 45.0, FN 15.0
Sensitivity = 0.8611111044883728
Specificity = 0.9694707989692688


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype